In [1]:
from huggingface_hub import hf_hub_download
import torch
from transformers import TimeSeriesTransformerForPrediction, TimeSeriesTransformerConfig, TimeSeriesTransformerModel

In [9]:
import torch
data = torch.rand(12,10,3)
# print(data)
print(data[:,-1,:].unsqueeze(1).shape)
data = torch.cat((data, data[:,-1,:].unsqueeze(1)),dim=1)
print(data)


torch.Size([12, 1, 3])
tensor([[[4.2641e-01, 3.6268e-02, 2.7871e-01],
         [4.0906e-01, 7.8652e-01, 3.5934e-01],
         [8.4251e-01, 6.3028e-01, 3.6465e-01],
         [4.8450e-01, 8.1085e-01, 1.3139e-01],
         [3.0891e-01, 5.0767e-01, 9.7587e-01],
         [4.7287e-01, 3.8401e-01, 8.9538e-01],
         [7.0356e-02, 8.8449e-01, 1.7575e-01],
         [5.4985e-01, 9.0537e-01, 2.7571e-01],
         [6.8929e-01, 9.9291e-01, 7.4718e-01],
         [8.7458e-01, 6.0551e-01, 3.3010e-01],
         [8.7458e-01, 6.0551e-01, 3.3010e-01]],

        [[3.6944e-01, 3.7984e-01, 7.2225e-01],
         [2.2228e-03, 4.7676e-02, 6.2349e-01],
         [1.9012e-01, 3.1139e-01, 8.4146e-01],
         [4.4738e-01, 6.4928e-02, 6.4491e-01],
         [5.7531e-03, 6.7091e-01, 8.1593e-01],
         [9.1024e-01, 3.6144e-01, 1.2201e-01],
         [5.4872e-01, 7.9318e-01, 6.2732e-01],
         [3.8312e-01, 4.8178e-01, 8.7700e-02],
         [3.2606e-01, 8.9937e-01, 2.7801e-01],
         [4.5224e-01, 9.0023e-01, 6

In [13]:
%pip install json

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import glob
import math
%matplotlib notebook
%matplotlib inline
import matplotlib.pyplot as plt
import scipy.io
from scipy import stats
import os
import copy
import time
import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import DataLoader, Dataset, Subset, random_split
from torchvision.transforms import ToTensor
#from torch.utils.tensorboard import SummaryWriter
import torch.optim as optim
from sklearn.model_selection import train_test_split
from tqdm.autonotebook import tqdm
import sys
# sys.path.append('/scratch/xc1490/projects/tmp/python_packages')
import wandb
wandb.__version__
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


/ext3/miniconda3/lib/python3.11/site-packages/IPython/core/magics/pylab.py:162: UserWarning: pylab import has clobbered these variables: ['copy']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [3]:
# config
PROJECT_PATH = os.path.dirname(os.getcwd())
FRAME_RATE = 60 # 60 frames/sec
MAX_HISTORY_TIME = 10
MAX_PREDICTION_TIME = 10
HISTORY_TIME = 1.0
PREDICTION_TIME = 0.1
HISTORY_LENGTH = int(HISTORY_TIME*FRAME_RATE)
PREDICTION_LENGTH = int(PREDICTION_TIME*FRAME_RATE)
MAX_HISTORY_LENGTH = MAX_HISTORY_TIME*FRAME_RATE
MAX_PREDICTION_LENGTH = MAX_PREDICTION_TIME*FRAME_RATE
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("device: ", DEVICE)
TOTAL_FEATURE_NAMES = ['head_x','head_y','head_z','head_r_sin','head_r_cos','head_p_sin','head_p_cos','head_y_sin',\
'head_y_cos','head_rx','head_ry','head_rz']
DEFAULT_FEATURE_NAMES = ['head_x','head_y','head_z','head_r_sin','head_r_cos','head_p_sin','head_p_cos','head_y_sin',\
'head_y_cos']
XYZ_FEATURE_NAMES = ['head_x', 'head_y', 'head_z']
CUSTOM_FEATURES = ['head_x', 'head_y','head_z']
ONE_FEATURE = ['head_x']
SC_FEATURE_NAMES = ['head_r_sin','head_r_cos','head_p_sin','head_p_cos','head_y_sin','head_y_cos']
RPY_FEATURE_NAMES = ['head_rx','head_ry','head_rz']
ANGLE_FEATURE_NAMES = ['head_r_cos','head_p_sin','head_p_cos','head_y_sin','head_y_cos','head_rx','head_ry','head_rz']
FEATURE_NAMES = XYZ_FEATURE_NAMES
FEATURE_INDEX = [TOTAL_FEATURE_NAMES.index(x) for x in FEATURE_NAMES]
DEFAULT_FEATURE_SIZE = len(DEFAULT_FEATURE_NAMES)
FEATURE_SIZE = len(FEATURE_NAMES)
BATCH_SIZE = 16

device:  cpu


In [4]:
class FoVDataset(Dataset):
    def __init__(self, x_data, y_data, feature_idx):
        self.feature_idx = feature_idx
        self.x_data = x_data[:,:,feature_idx]
        self.y_data = y_data[:,:,feature_idx]

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        x = torch.tensor(self.x_data[idx])
        y = torch.tensor(self.y_data[idx])
        return x,y

In [5]:
# load data
processed_data_path = f'{PROJECT_PATH}/processed_data'
x_train = np.loadtxt(f'{processed_data_path}/x_train_{HISTORY_TIME}_{PREDICTION_TIME}.csv', dtype='float32', delimiter=',').reshape((-1,HISTORY_LENGTH,DEFAULT_FEATURE_SIZE))
y_train = np.loadtxt(f'{processed_data_path}/y_train_{HISTORY_TIME}_{PREDICTION_TIME}.csv', dtype='float32', delimiter=',').reshape((-1,PREDICTION_LENGTH,DEFAULT_FEATURE_SIZE))
x_val = np.loadtxt(f'{processed_data_path}/x_val_{HISTORY_TIME}_{PREDICTION_TIME}.csv', dtype='float32', delimiter=',').reshape((-1,HISTORY_LENGTH,DEFAULT_FEATURE_SIZE))
y_val = np.loadtxt(f'{processed_data_path}/y_val_{HISTORY_TIME}_{PREDICTION_TIME}.csv', dtype='float32', delimiter=',').reshape((-1,PREDICTION_LENGTH,DEFAULT_FEATURE_SIZE))
x_test = np.loadtxt(f'{processed_data_path}/x_test_{HISTORY_TIME}_{PREDICTION_TIME}.csv', dtype='float32', delimiter=',').reshape((-1,HISTORY_LENGTH,DEFAULT_FEATURE_SIZE))
y_test = np.loadtxt(f'{processed_data_path}/y_test_{HISTORY_TIME}_{PREDICTION_TIME}.csv', dtype='float32', delimiter=',').reshape((-1,PREDICTION_LENGTH,DEFAULT_FEATURE_SIZE))
mean_std = np.loadtxt(f'{processed_data_path}/xyz_mean_std_{HISTORY_TIME}_{PREDICTION_TIME}.csv', dtype='float32', delimiter=',').reshape((3, -1))

In [6]:
# create dataset and dataloader
feature_names = FEATURE_NAMES
feature_idx = FEATURE_INDEX
x_train = x_train[:,:,feature_idx]
y_train = y_train[:,:,feature_idx]
x_val = x_val[:,:,feature_idx]
y_val = y_val[:,:,feature_idx]
x_test = x_test[:,:,feature_idx]
y_test = y_test[:,:,feature_idx]
train_data = FoVDataset(x_train, y_train, feature_idx)
val_data = FoVDataset(x_val, y_val, feature_idx)
test_data = FoVDataset(x_test, y_test, feature_idx)
train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=False)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)

In [7]:
configuration = TimeSeriesTransformerConfig(
    prediction_length = PREDICTION_LENGTH,
    context_length = HISTORY_LENGTH,
    input_size = FEATURE_SIZE,
    lags_sequence=[0],
    num_dynamic_real_features=FEATURE_SIZE
)

In [8]:
file = hf_hub_download(
    repo_id="hf-internal-testing/tourism-monthly-batch", filename="train-batch.pt", repo_type="dataset"
)
batch = torch.load(file)

model = TimeSeriesTransformerForPrediction.from_pretrained(
    "huggingface/time-series-transformer-tourism-monthly"
)

mymodel = TimeSeriesTransformerForPrediction(configuration)
# mymodel = TimeSeriesTransformerModel(configuration)
# during training, one provides both past and future values
# as well as possible additional features
# outputs = model(
#     past_values=batch["past_values"],
#     past_time_features=batch["past_time_features"],
#     past_observed_mask=batch["past_observed_mask"],
#     static_categorical_features=batch["static_categorical_features"],
#     static_real_features=batch["static_real_features"],
#     future_values=batch["future_values"],
#     future_time_features=batch["future_time_features"],
# )


/ext3/miniconda3/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [9]:
def my_loss(output, target, feature_names):
    # compute loss for each feature
    # output/target: [N, seq_len, 9]
    # change sin cos back to angle ?
    if 'head_r_sin' in feature_names:
        feature_idx = feature_names.index('head_r_sin')
        output_rx = torch.atan2(output[:,:,feature_idx], output[:,:,feature_idx+1]).unsqueeze(2)
        target_rx = torch.atan2(target[:,:,feature_idx], target[:,:,feature_idx+1]).unsqueeze(2)
        output = torch.cat((output,output_rx),-1)
    if 'head_p_sin' in feature_names:
        feature_idx = feature_names.index('head_p_sin')
        output_ry = torch.atan2(output[:,:,feature_idx], output[:,:,feature_idx+1]).unsqueeze(2)
        target_ry = torch.atan2(target[:,:,feature_idx], target[:,:,feature_idx+1]).unsqueeze(2)
        output = torch.cat((output,output_ry),-1)
    if 'head_y_sin' in feature_names:
        feature_idx = feature_names.index('head_y_sin')
        output_rz = torch.atan2(output[:,:,feature_idx], output[:,:,feature_idx+1]).unsqueeze(2)
        target_rz = torch.atan2(target[:,:,feature_idx], target[:,:,feature_idx+1]).unsqueeze(2)
        output = torch.cat((output,output_rz),-1)
    mse_loss = torch.mean((output - target) ** 2, [0,1])
    pearsonr = stats.pearsonr(output.detach().cpu().flatten(), target.detach().cpu().flatten())
    return mse_loss, pearsonr

def train(device, model: nn.Module, data_loader, optimizer, step, feature_names, plot_flag = False):
    progress_bar = tqdm(data_loader)
    model.train() # turn on train mode
    feature_size = len(feature_names)
    num_batches = len(data_loader)
    log_interval = num_batches // 5
    loss_names = list(map(lambda x: x+'_loss', feature_names))
    start_time = time.time()
    total_loss = 0.
    return_loss = 0.
    sep_return_loss = np.zeros((1,feature_size))
    loss_dict_train = {}
    for batch_idx, (data, targets) in enumerate(progress_bar):
#         print("batch index: ", batch_idx)
        data = data.to(device)
        targets = targets.to(device)
        # with torch.cuda.amp.autocast():
        hist_pe = torch.arange(0,data.shape[1]).repeat(FEATURE_SIZE,1).T.repeat(data.shape[0],1).reshape(data.shape)
#         hist_pe = torch.arange(0,data.shape[1]).repeat(data.shape[0],1)
        pred_pe = torch.arange(0,targets.shape[1]).repeat(FEATURE_SIZE,1).T.repeat(targets.shape[0],1).reshape(targets.shape)
#         pred_pe = torch.arange(0,targets.shape[1]).repeat(targets.shape[0],1)
        
        output = model(past_values=data,
                         past_time_features=hist_pe,
                         past_observed_mask=torch.ones(data.shape),
                         future_values=targets,
                         future_time_features=pred_pe
                        )
        optimizer.zero_grad()
#         sep_mse_loss, train_pearsonr = my_loss(output, targets, feature_names)
#         loss = torch.sum(sep_mse_loss[:feature_size])
        loss = output.loss
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
        optimizer.step()
        step += 1
        total_loss += loss.item()
        return_loss += loss.item()
        sep_return_loss += loss.item()
        progress_bar.set_postfix_str(f"training loss={loss.item():.4e}|avg training loss={total_loss/(batch_idx+1):.4e}")
        loss_dict_train['training loss'] = loss.item()
#         for name_count, loss_name in enumerate(loss_names):
#             loss_dict_train[loss_name] = sep_return_loss[name_count]
#         loss_dict_train['avg training loss'] = total_loss/(batch_idx+1)
#         loss_dict_train['train pearsonr'] = train_pearsonr
        
        if use_wandb:
            wandb.log(loss_dict_train) 
        if batch_idx % log_interval == 0 and batch_idx > 0:
            lr = scheduler.get_last_lr()[0]
            ms_per_batch = (time.time() - start_time) * 1000 / log_interval
            cur_loss = total_loss / log_interval
            print(f'| epoch {epoch:3d} | {batch_idx:5d}/{num_batches:5d} batches | '
                  f'lr {lr:02.5f} | ms/batch {ms_per_batch:5.5f} | '
                  f'loss {cur_loss:5.5f}')
            total_loss = 0
            start_time = time.time()
            if plot_flag == True:
                # training prediction
                print ('training prediction')
                visualize_data(data.detach().cpu().numpy(), targets.detach().cpu().numpy(), output.detach().cpu().numpy())
    return return_loss/(batch_idx+1), sep_return_loss/(batch_idx+1), train_pearsonr


def validate(device, model: nn.Module, dataloader: DataLoader, feature_names, plot_flag = False):
    feature_size = len(feature_names)
    loss_names = list(map(lambda x: x+'_loss', feature_names))
    model.eval()
    total_loss = 0.
    sep_total_loss = np.zeros((1,feature_size))
    loss_dict_valid = {}
    iter_count = 0
    with torch.no_grad():
        for (data, targets) in dataloader:
            data = data.to(device) # [N, seq_len, feature_size]
            targets = targets.to(device) # [N, seq_len, feature_size]
#             output = model(data)
            hist_pe = torch.arange(0,data.shape[1]).repeat(FEATURE_SIZE,1).T.repeat(data.shape[0],1).reshape(data.shape)
            pred_pe = torch.arange(0,targets.shape[1]).repeat(FEATURE_SIZE,1).T.repeat(targets.shape[0],1).reshape(targets.shape)
            outputs = model.generate(
                past_values=data,
                past_time_features=hist_pe,
                past_observed_mask=torch.ones(data.shape),
                future_values=targets,
                future_time_features=pred_pe
            )
            print(outputs)
#             if plot_flag and iter_count == 0:
#                 print ('validation prediction')
#                 visualize_data(data.detach().cpu().numpy(), targets.detach().cpu().numpy(), output.detach().cpu().numpy())
#                 if use_wandb:
#                     wandb.log({"validation plot": fig})
#             sep_valid_loss, valid_pearsonr = my_loss(output, targets, feature_names)
#             total_loss += torch.sum(sep_valid_loss)
#             sep_total_loss += sep_valid_loss.detach().cpu().numpy()
#             #import pdb;pdb.set_trace()
#             for name_count, loss_name in enumerate(loss_names):
#                 loss_dict_valid[loss_name] = sep_valid_loss[name_count]
#             #loss_dict_valid['valid loss'] = sep_valid_loss
#             loss_dict_valid['valid pearsonr'] = valid_pearsonr[0]
#             iter_count += 1
#             if use_wandb:
#                 wandb.log(loss_dict_valid) 
    return total_loss/(len(dataloader) - 1), sep_total_loss/(len(dataloader) - 1), valid_pearsonr



In [10]:
#training hyperparams
in_seq_len = HISTORY_LENGTH
out_seq_len = PREDICTION_LENGTH

n_heads = 3 ##4
head_dim = 3 #32 # dimension of each head, not total
dim_val = 9 #16#n_heads*head_dim # embedding dimension, all heads together
feature_size = FEATURE_SIZE
lr = 0.005
tf_rate = 0.5
epochs = 200
n_decoder_layers = 2
n_encoder_layers = 2
batch_size = BATCH_SIZE
n_batches = len(train_dataloader)
model = mymodel
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)
#keep track of loss for graph
train_mse_losses = []
val_mse_losses = []
test_mse_losses = []
sep_train_mse_losses = []
sep_val_mse_losses = []
sep_test_mse_losses = []
train_pearsonr_arr = []
val_pearsonr_arr = []
test_pearsonr_arr = []

In [11]:
# Trainig
#build live matplotlib fig


#plt.ion()
#fig.show()
#fig.canvas.draw()
epochs = 20
LOAD_MODEL = False
use_wandb = False
if LOAD_MODEL:
    load_ckpt(f"{PROJECT_PATH}/checkpoints/batch_{BATCH_SIZE}_{HISTORY_TIME}_{PREDICTION_TIME}_ckpts.pt".format(os.getcwd(), epochs, batch_size), model, optimizer)
lr = 0.1
# scaler = torch.cuda.amp.GradScaler()
# print(scaler)
best_val_loss = float('inf')
#writer = SummaryWriter("run/loss_plot")
step = 0
for epoch in tqdm(range(1, epochs+1)):
    print(f'Epoch #{epoch}')
    epoch_start_time = time.time()
    train_loss, sep_train_loss, train_pearsonr = train(DEVICE, model, train_dataloader, optimizer, step, feature_names, plot_flag=True if epoch % 2 == 1 else False)
#     print(train_loss)
    train_mse_losses.append(train_loss)
    sep_train_mse_losses.append(sep_train_loss)
    mean_loss = sum(train_mse_losses)/len(train_mse_losses)
    print("validation")
    val_loss, sep_val_loss, val_pearsonr = validate(DEVICE, model, val_dataloader, feature_names, plot_flag=True if epoch % 2 == 1 else False)
    val_mse_losses.append(val_loss)
    sep_val_mse_losses.append(sep_val_loss)
    test_loss, sep_test_loss, test_pearsonr = validate(DEVICE, model, test_dataloader, feature_names, plot_flag=True if epoch % 2 == 1 else False)
    test_mse_losses.append(test_loss)
    sep_test_mse_losses.append(sep_test_loss)
    elapsed = time.time() - epoch_start_time
    print('-' * 89)
    print(f'| end of epoch {epoch:3d} | time: {elapsed:5.2f}s | '
        f'valid loss {val_loss:5.4f} | mean loss {mean_loss:8.4f}')
    print('-' * 89)
    if epoch % 2 == 1:
        fig, ax = plt.subplots(1,2, figsize=(12,4))
        #ax.plot([i.detach().cpu().numpy() for i in test_losses], label='train loss')
        ax[0].plot([i.detach().cpu().numpy() for i in val_mse_losses[5:]] , label='validation loss')
        ax[0].plot([i  for i in train_mse_losses[5:]] , label='train loss')
        ax[0].set_title("Losses")

        ax[0].legend()
        ax[1].plot([i.detach().cpu().numpy() for i in test_mse_losses], label='test loss')
        ax[1].set_title("Losses")
        ax[1].legend()
        #fig.canvas.draw()
        plt.show()
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        # save model
#         save_ckpt("{}/checkpoints/epoch_{}_ckpts.pt".format(os.getcwd(), epoch), model, optimizer,epochs, val_loss)
        save_dict = {
            'epoch': epoch,
            'train_mse_losses': train_mse_losses,
            'val_mse_losses': val_mse_losses
        }
        save_ckpt(f"{PROJECT_PATH}/checkpoints", model, optimizer, save_dict)

    # scheduler.step(mean_loss)
    scheduler.step()
# load_ckpt("/content/drive/MyDrive/checkpoints.pt", model, optimizer)


  0%|          | 0/20 [00:00<?, ?it/s]

Epoch #1


  0%|          | 0/423 [00:00<?, ?it/s]

| epoch   1 |    84/  423 batches | lr 0.00500 | ms/batch 2363.54075 | loss -0.33710
training prediction


NameError: name 'visualize_data' is not defined

In [ ]:
16*60

960

In [ ]:
mymodel.config

TimeSeriesTransformerConfig {
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "attention_dropout": 0.1,
  "cardinality": [
    0
  ],
  "context_length": 60,
  "d_model": 64,
  "decoder_attention_heads": 2,
  "decoder_ffn_dim": 32,
  "decoder_layerdrop": 0.1,
  "decoder_layers": 2,
  "distribution_output": "student_t",
  "dropout": 0.1,
  "embedding_dimension": [
    0
  ],
  "encoder_attention_heads": 2,
  "encoder_ffn_dim": 32,
  "encoder_layerdrop": 0.1,
  "encoder_layers": 2,
  "feature_size": 24,
  "init_std": 0.02,
  "input_size": 3,
  "is_encoder_decoder": true,
  "lags_sequence": [
    0,
    0,
    0,
    0,
    0,
    0
  ],
  "loss": "nll",
  "model_type": "time_series_transformer",
  "num_dynamic_real_features": 0,
  "num_parallel_samples": 100,
  "num_static_categorical_features": 0,
  "num_static_real_features": 0,
  "num_time_features": 0,
  "prediction_length": 6,
  "scaling": "mean",
  "transformers_version": "4.38.2",
  "use_cache": true
}

## model.eval()
total_loss, sep_loss, pearsonr = validate(DEVICE, model, val_dataloader,XYZ_FEATURE_NAMES)

In [ ]:
batch["past_values"].shape #[batch_size, num_batch, feature_size]?

torch.Size([64, 61])

In [ ]:
batch["past_time_features"]

tensor([[[ 0.1364,  2.2175],
         [ 0.2273,  2.2201],
         [ 0.3182,  2.2227],
         ...,
         [-0.0455,  2.3483],
         [ 0.0455,  2.3502],
         [ 0.1364,  2.3522]],

        [[ 0.1364,  2.1847],
         [ 0.2273,  2.1875],
         [ 0.3182,  2.1903],
         ...,
         [-0.0455,  2.3243],
         [ 0.0455,  2.3263],
         [ 0.1364,  2.3284]],

        [[ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         ...,
         [-0.2273,  0.6990],
         [-0.1364,  0.7782],
         [-0.0455,  0.8451]],

        ...,

        [[ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         ...,
         [ 0.4091,  1.5563],
         [ 0.5000,  1.5682],
         [-0.5000,  1.5798]],

        [[-0.4091,  1.7993],
         [-0.3182,  1.8062],
         [-0.2273,  1.8129],
         ...,
         [ 0.5000,  2.0828],
         [-0.5000,  2.0864],
         [-0.4091,  2.0899]],

        [[ 0.5000,  1.1139],
       

In [ ]:
batch["past_observed_mask"]

tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [0., 0., 0.,  ..., 1., 1., 1.],
        ...,
        [0., 0., 0.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]])

In [ ]:
batch["static_categorical_features"].shape

torch.Size([64, 1])

In [ ]:
batch["static_real_features"].shape

torch.Size([64, 1])

In [ ]:
batch["future_time_features"].shape

torch.Size([64, 24, 2])

In [ ]:
batch["future_values"].shape

torch.Size([64, 24])

TimeSeriesTransformerConfig {
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "attention_dropout": 0.1,
  "cardinality": [
    0
  ],
  "context_length": 60,
  "d_model": 64,
  "decoder_attention_heads": 2,
  "decoder_ffn_dim": 32,
  "decoder_layerdrop": 0.1,
  "decoder_layers": 2,
  "distribution_output": "student_t",
  "dropout": 0.1,
  "embedding_dimension": [
    0
  ],
  "encoder_attention_heads": 2,
  "encoder_ffn_dim": 108,
  "encoder_layerdrop": 0.1,
  "encoder_layers": 2,
  "feature_size": 9,
  "init_std": 0.02,
  "input_size": 3,
  "is_encoder_decoder": true,
  "lags_sequence": [
    0
  ],
  "loss": "nll",
  "model_type": "time_series_transformer",
  "num_dynamic_real_features": 0,
  "num_parallel_samples": 100,
  "num_static_categorical_features": 0,
  "num_static_real_features": 0,
  "num_time_features": 0,
  "prediction_length": 6,
  "scaling": "mean",
  "transformers_version": "4.38.2",
  "use_cache": true
}